In [4]:
!pip install fake_useragent
!pip install re

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=42f6ebb5a949f6308d1cfe1984258665fe1b8c57dd6fcb08cdbe169f547178e0
  Stored in directory: c:\users\suma marri\appdata\local\pip\cache\wheels\ae\e7\76\7dd44644d065268ab0e1b4fa2e802fa4bb0157717b7d6c6d92
Successfully built fake-useragent


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


## Initialize

In [5]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re
import time
import random
import datetime

In [6]:
ua = UserAgent()

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\Users\Suma Marri\anaconda3\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "C:\Users\Suma Marri\anaconda3\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


## Find the total number of pages

In [7]:
def total_pages(url):
    useragent = ua.random    # randomly choose a user-agent
    headers = {'User-Agent':useragent}
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content,'lxml')
    pages = soup.find('div',class_="pagination").find_all('li')[-1].find('a').get_text()
    pages = int(pages)    # 'pages' is the total number of pages
    return pages

## Function to parse each item's page

In [8]:
def help_retrieve(dic,key):
    try:
        return dic[key]
    except KeyError:
        return np.NaN

In [9]:
def parse_item(url):
    item_url = 'https://www.winemag.com/buying-guide' + url
    useragent = ua.random    # randomly choose a user-agent
    headers = {'User-Agent':useragent}
    response = requests.get(item_url,headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content,'lxml')
        try:
            description = soup.find('p',class_='description').get_text()
        except AttributeError:
            description = np.NaN
        try:
            taster = soup.find('span',class_='taster-area').get_text()
        except AttributeError:
            taster = np.NaN

        info_list = soup.find_all('li',class_='row')
        info_dict = dict()

        for element in info_list:
            tag = element.find_all('div')[0].get_text().strip()
            value = element.find_all('div')[1].get_text().strip()
            info_dict[tag] = value

        designation = help_retrieve(info_dict,"Designation")
        variety = help_retrieve(info_dict,"Variety")
        appellation = help_retrieve(info_dict,"Appellation")
        winery = help_retrieve(info_dict,"Winery")
        alcohol = help_retrieve(info_dict,"Alcohol")
        bottle_size = help_retrieve(info_dict,"Bottle Size")
        category = help_retrieve(info_dict,"Category")
        importer = help_retrieve(info_dict,"Importer")
        date_published = help_retrieve(info_dict,"Date Published")
        user_avg_rating = help_retrieve(info_dict,"User Avg Rating")

        if user_avg_rating == 'Not rated yet [Add Your Review]':
            user_avg_rating = np.NaN

        related_items = soup.find_all('li',class_='review-item')
        related_items = [element.find('a').get('data-review-id') for element in related_items]    # related_items holds a list of related items' ids

        return (description,taster,designation,variety,appellation,winery,alcohol,bottle_size,\
                category,importer,date_published,user_avg_rating,related_items)
    else:
        print(f"{item_url} request failed! Status code: {response.status_code}. Skip it.")
        raise RuntimeError

## Function to parse each page

In [10]:
# only store the ending of each item's url to save memory, because the former parts are the same for all
pattern = re.compile(r'https://www.winemag.com/buying-guide(?P<ending>.*)')

# request and parse each page
# return a list with items in this page as tuples inside
def parse_page(base_url,i):
    url = re.sub(r"page=\d+",f"page={i}",base_url)  
    useragent = ua.random    # randomly choose a user-agent
    headers = {'User-Agent':useragent}
    response = requests.get(url,headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content,'lxml')
        li = soup.find_all('li',class_="review-item")
        page = list()

        for element in li:
            try:
                name = element.find('h3').get_text()
                rating = element.find('span',class_="rating").find('strong').get_text()
                price = element.find('span',class_="price").get_text()
                price = price.replace('$','')
            except AttributeError:
                print(f"Some items are missed from page {i}: {url}. Skip it.")
                continue
            try:
                price = float(price)
            except ValueError:
                price = np.NaN
            item_url = element.find('a',class_="review-listing row").get('href')
            item_url = pattern.search(item_url).group('ending')
            item_id = element.find('a',class_="review-listing row").get('data-review-id')
            try:
                details = parse_item(item_url)
            except RuntimeError:
                details = (np.NaN,)*13
            all_info = (item_id,name,rating,price,item_url) + details
            page.append(all_info)
            
        return page
    else:
        print(f"Page #{i} request failed! Status code: {response.status_code}. Skip it.")
        raise RuntimeError()

## Scrape all the pages

In [11]:
def scrape_all(url):
    result = list()
    start = datetime.datetime.now()
    pages = total_pages(url)

    for i in range(1,pages+1):
        try:
            result.extend(parse_page(url,i))
            time.sleep(random.random())
        except RuntimeError:
            pass
        except:
            print(r"No worry! I'm still working :)")
        finally:
            if i%10 == 0:
                print(f"Process overview: {i} pages have been scraped...")

    end = datetime.datetime.now()
    interval = (end - start).total_seconds()
    hour = int(interval // 3600)
    minute = int((interval % 3600) // 60)
    second = int((interval % 3600) % 60)
    print(f"For base url: {url}")
    print(f"Scraping has been done in {hour}h {minute}min {second}s. Congrats!")
    
    return result

In [12]:
def scrape_url_list(urls, file_name="raw_data.csv"):
    result = list()
    
    for url in urls:
        try:
            print(f"Begin scraping from base url: {url}")
            result.extend(scrape_all(url))
        except:
            print(f"Scraping from base url: {url} FAILED! Going to the next one...")
            continue
        
    df = pd.DataFrame(result,columns=['id','name','rating','price','item_url','description','taster',\
                                      'designation','variety','appellation','winery','alcohol','bottle_size',\
                                        'category','importer','date_published','user_avg_rating','related_items'])
    df.to_csv(file_name,encoding='utf-8')
    
    return df

In [13]:
urls = ['https://www.winemag.com/?s=&rating=98.0-*&drink_type=wine&page=1&search_type=reviews',\
       'https://www.winemag.com/?s=&rating=94.0-97.99&drink_type=wine&page=1&search_type=reviews',\
       'https://www.winemag.com/?s=&rating=80.0-82.99&drink_type=wine&page=1&search_type=reviews',\
       'https://www.winemag.com/?s=&rating=90.0-93.99&price=16.0-25.99,26.0-40.99&drink_type=wine&page=1&search_type=reviews',\
        'https://www.winemag.com/?s=&rating=83.0-86.99&price=41.0-60.99,61.0-75.99,76.0-99.99,100.0-199.99,200.0-*&drink_type=wine&page=1&search_type=reviews',\
       'https://www.winemag.com/?s=&rating=90.0-93.99&price=1.0-15.99,41.0-60.99,61.0-75.99,76.0-99.99,100.0-199.99,200.0-*&drink_type=wine&page=1&search_type=reviews',\
        'https://www.winemag.com/?s=&rating=83.0-86.99&price=1.0-15.99&drink_type=wine&page=1&search_type=reviews',\
       'https://www.winemag.com/?s=&rating=87.0-89.99&price=1.0-15.99,26.0-40.99,41.0-60.99&drink_type=wine&page=1&search_type=reviews',\
        'https://www.winemag.com/?s=&rating=83.0-86.99&price=26.0-40.99&drink_type=wine&page=1&search_type=reviews',\
       'https://www.winemag.com/?s=&rating=87.0-89.99&price=16.0-25.99,61.0-75.99,76.0-99.99,100.0-199.99,200.0-*&drink_type=wine&page=1&search_type=reviews',\
       'https://www.winemag.com/?s=&rating=83.0-86.99&price=16.0-25.99&drink_type=wine&page=1&search_type=reviews']

## Begin scraping!

In [ ]:
df = scrape_url_list(urls)

Begin scraping from base url: https://www.winemag.com/?s=&rating=98.0-*&drink_type=wine&page=1&search_type=reviews
Process overview: 10 pages have been scraped...
Some items are missed from page 13: https://www.winemag.com/?s=&rating=98.0-*&drink_type=wine&page=13&search_type=reviews. Skip it.
https://www.winemag.com/buying-guide/kistler-2015-stone-flat-vineyard-chardonnay-sonoma-coast/ request failed! Status code: 502. Skip it.
Process overview: 20 pages have been scraped...
Process overview: 30 pages have been scraped...
Some items are missed from page 32: https://www.winemag.com/?s=&rating=98.0-*&drink_type=wine&page=32&search_type=reviews. Skip it.
Some items are missed from page 33: https://www.winemag.com/?s=&rating=98.0-*&drink_type=wine&page=33&search_type=reviews. Skip it.
Some items are missed from page 33: https://www.winemag.com/?s=&rating=98.0-*&drink_type=wine&page=33&search_type=reviews. Skip it.
Some items are missed from page 33: https://www.winemag.com/?s=&rating=98.0

Process overview: 480 pages have been scraped...
Process overview: 490 pages have been scraped...
Process overview: 500 pages have been scraped...
Process overview: 510 pages have been scraped...
https://www.winemag.com/buying-guide/perrier-jouet-2004-belle-epoque-blanc-de-blancs-brut-chardonnay-champagne/ request failed! Status code: 502. Skip it.
No worry! I'm still working :)
Process overview: 520 pages have been scraped...
https://www.winemag.com/buying-guide/lambert-2013-the-family-tree-shiraz-barossa-valley/ request failed! Status code: 502. Skip it.
https://www.winemag.com/buying-guide/quinta-da-zaralhoa-2011-red-douro/ request failed! Status code: 502. Skip it.
Process overview: 530 pages have been scraped...
No worry! I'm still working :)
Process overview: 540 pages have been scraped...
Process overview: 550 pages have been scraped...
Page #558 request failed! Status code: 502. Skip it.
Process overview: 560 pages have been scraped...
Process overview: 570 pages have been scra

Process overview: 150 pages have been scraped...
https://www.winemag.com/buying-guide/ricardo-santos-2009-semillon-mendoza/ request failed! Status code: 502. Skip it.
Process overview: 160 pages have been scraped...
No worry! I'm still working :)
https://www.winemag.com/buying-guide/bodegas-farina-2009-dama-toro-italian-white-malvasia-toro/ request failed! Status code: 502. Skip it.
Process overview: 170 pages have been scraped...
Process overview: 180 pages have been scraped...
Process overview: 190 pages have been scraped...
Process overview: 200 pages have been scraped...
No worry! I'm still working :)
No worry! I'm still working :)
Process overview: 210 pages have been scraped...
No worry! I'm still working :)
No worry! I'm still working :)
No worry! I'm still working :)
No worry! I'm still working :)
Process overview: 220 pages have been scraped...
Process overview: 230 pages have been scraped...
Process overview: 240 pages have been scraped...
Process overview: 250 pages have bee

In [ ]:
df